Goal: Create a daily NAO index using the CPC methodology 
=====

Author: [Ray Bell](https://github.com/raybellwaves)

Use the time period of the [SubX](http://iridl.ldeo.columbia.edu/SOURCES/.Models/.SubX/) hindcast: 1999-2016.
For a saninty check make sure that 1999-2016 is indeed the hindcast period:

In [ ]:
import xarray as xr
remote_data = xr.open_dataset('http://iridl.ldeo.columbia.edu/SOURCES/.Models'\
                              '/.SubX/.RSMAS/.CCSM4/.hindcast/.zg/dods')
print(remote_data.coords['S'])

The data used here is geopotential height at 500 hPa ($Z_{500}$) from ERA-Interim. This data was simply copied from the University of Reading archive. You can however use the [API](https://software.ecmwf.int/wiki/display/CKB/How+to+download+data+via+the+ECMWF+WebAPI) to download the data from ECMWF.

Two files exists in this directory `a.nc` (43 Mb) and `b.nc`. `a.nc` is the monthly mean of $Z_{500}$ January 1996 - December 2016. `b.nc` is daily $Z_{500}$ January 1st 1996 - December 31st 2016. The data has been interpolated to 1 degree to match the SubX data (360x181) and cut to 20°N-90°N for the analysis. The refernce scripts for the data processing beforehand are in the directory `pre_proc`. DO THIS!

CPC's methodology for the daily NAO index can be found [here](http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/history/method.shtml) with some more information [here](http://www.cpc.ncep.noaa.gov/data/teledoc/telepatcalc.shtml) and the procedure is based on [Barston and Livezey (1987)](https://journals.ametsoc.org/doi/pdf/10.1175/1520-0493%281987%29115%3C1083%3ACSAPOL%3E2.0.CO%3B2).

They use the region of 20°N-90°N and monthly standardized $Z_{500}$ anomalies.

They calculate the 10 leading EOFs for each calendar month centred on that month with a window of 3 months
e.g. February is based on January-February-March. January is calculated as ... December is calcualted as ...

Enter a start-time an end-time for which to calculate the daily NAO index
e.g. lets look at the large negativate NAO of winter 2008/2009:

In [ ]:
stime = '2009-02-01-T12:00:00'
etime = '2009-02-14-T12:00:00'

Now extend for the winter months (DJF)

In [ ]:
stime = '2008-12-01-T12:00:00'
etime = '2009-02-28-T12:00:00'